In [1]:
from __future__ import print_function

import tensorflow as tf
import os

# Dataset Parameters - CHANGE HERE
MODE = 'folder' # or 'file', if you choose a plain text file (see above).
DATASET_PATH = 'flower_photos/' # the dataset file or root folder path.
MODEL_PATH = 'saved/my_tf_model'

# Image Parameters
N_CLASSES = 5 # CHANGE HERE, total number of classes
IMG_HEIGHT = 64 # CHANGE HERE, the image height to be resized to
IMG_WIDTH = 64 # CHANGE HERE, the image width to be resized to
CHANNELS = 3 # The 3 color channels, change to 1 if grayscale

classes = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Create model
def conv_net(x, n_classes, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):

        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Convolution Layer with 32 filters and a kernel size of 5
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # Fully connected layer (in contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
#        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)
        # Because 'softmax_cross_entropy_with_logits' already apply softmax,
        # we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out

    return out

In [ ]:
INPUT_PATH = 'flower_photos/tulips/100930342_92e8746431_n.jpg'
# Read images from disk
image = tf.read_file(INPUT_PATH)
image = tf.image.decode_jpeg(image, channels=CHANNELS)

# Resize images to a common size
image = tf.image.resize_images(image, [IMG_HEIGHT, IMG_WIDTH])

# Normalize
image = image * 1.0/127.5 - 1.0

X_input = tf.reshape(image, [1, IMG_HEIGHT, IMG_WIDTH, CHANNELS])
#X = image.reshape(1, IMG_HEIGHT, IMG_WIDTH, CHANNELS)

In [ ]:
#tf.reset_default_graph()
#tf.get_variable_scope().reuse_variables()

with tf.Session() as sess:
    logits_test = conv_net(X_input, N_CLASSES, reuse=tf.AUTO_REUSE, is_training=False)
    pred = tf.argmax(logits_test, 1)
    sess.run(tf.global_variables_initializer())
    new_saver = tf.train.import_meta_graph(MODEL_PATH + '.meta')
    new_saver.restore(sess, MODEL_PATH)
    probs, prediction = sess.run([logits_test, pred]) #, feed_dict={images: rand}))
    print('done')

INFO:tensorflow:Restoring parameters from saved/my_tf_model
done


In [ ]:
print("label: ", classes[prediction[0]])
print("probabilities: ", probs)